In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import category_encoders as ce

import warnings

warnings.filterwarnings("ignore")

def evaluate_categorical_encoders(X, y, encoders, cv_folds=5):
    """
    Evaluates different categorical encoders and returns the best one based on cross-validation scores.
    
    :param X: DataFrame with features, including categorical ones
    :param y: Target variable
    :param encoders: Dictionary of encoder names and their corresponding objects from category_encoders
    :param cv_folds: Number of folds for cross-validation
    :return: Name of the best encoder
    """
    results = {}
    
    # Define cross-validation strategy
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for encoder_name, encoder in encoders.items():
        # Create a pipeline with the encoder and XGBClassifier
        pipeline = make_pipeline(encoder, XGBClassifier(use_label_encoder=False))
        
        # Evaluate the pipeline
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='f1')
        
        # Store the mean score and print the results
        results[encoder_name] = scores.mean()
        print(f"{encoder_name}: Mean CV accuracy = {scores.mean():.4f} ± {scores.std():.4f}")
    
    # Identify the best encoder
    best_encoder_name = max(results, key=results.get)
    print(f"\nBest encoder: {best_encoder_name} with accuracy = {results[best_encoder_name]:.4f}")
    
    return best_encoder_name

def evaluate_model(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    best_f1 = 0.0

    model = XGBClassifier(random_state=42, enable_categorical=True)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=0, early_stopping_rounds=25)
    
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred)
            
    return f1


In [3]:
X = pd.read_csv('../../data/new_features/df_.csv')
y = pd.read_csv('../../data/new_features/y.csv') 

for col in X.filter(like='_binned').columns:
    X[col] = X[col].astype('category')

X.filter(like='_binned').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 1 columns):
 #   Column                                                      Non-Null Count  Dtype   
---  ------                                                      --------------  -----   
 0   ewm_home_team_goals_conceded_x_ewm_shoton_home_home_binned  3024 non-null   category
dtypes: category(1)
memory usage: 3.3 KB


In [4]:
evaluate_model(X, y)

0.6600138431251006

In [5]:
# Define the encoders to evaluate
encoders_to_evaluate = {
    'OneHotEncoder': ce.OneHotEncoder(drop_invariant=True),
    'OrdinalEncoder': ce.OrdinalEncoder(drop_invariant=True),
    'TargetEncoder': ce.TargetEncoder(drop_invariant=True, ),
    'BinaryEncoder': ce.BinaryEncoder(drop_invariant=True),
    'BaseNEncoder': ce.BaseNEncoder(drop_invariant=True),
    'CatBoostEncoder': ce.CatBoostEncoder(drop_invariant=True),
}

best_encoder = evaluate_categorical_encoders(X, y, encoders_to_evaluate)

OneHotEncoder: Mean CV accuracy = 0.5559 ± 0.0132
OrdinalEncoder: Mean CV accuracy = 0.5600 ± 0.0194
TargetEncoder: Mean CV accuracy = 0.5624 ± 0.0064
BinaryEncoder: Mean CV accuracy = 0.5641 ± 0.0258
BaseNEncoder: Mean CV accuracy = 0.5641 ± 0.0258
CatBoostEncoder: Mean CV accuracy = 0.5586 ± 0.0158

Best encoder: BinaryEncoder with accuracy = 0.5641
